# data import & prep

In [1]:
import sys
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd

from datetime import date
from datetime import datetime
import warnings 
warnings.filterwarnings(action='ignore')

# Data Overview & Import

## REGISTERED

In [2]:
registered_path = r'E:\Projects\canadacharities\data\raw\Registered'
registered_files = [f for f in listdir(registered_path) if isfile(join(registered_path, f))]

registered_path = registered_path + '\\'

financial_df1 = pd.read_csv(registered_path + 'Financial Section A_ B and C.csv', encoding='latin-1', dtype=str)
financial_df2 = pd.read_csv(registered_path + 'Financial Section D & Schedule 6.csv', encoding='latin-1', dtype=str)
identities_df = pd.read_csv(registered_path + 'Ident.csv', encoding='latin-1', dtype=str)

#setting up lookup tables
category_table = pd.read_csv(registered_path + '# Category_Sub-Category.csv', encoding='latin-1', dtype=str)
designation_table = pd.read_csv(registered_path + '# Designation.csv', encoding='latin-1', dtype=str)
programs_table = pd.read_csv(registered_path + '# Programs.csv', encoding='latin-1', dtype=str)

category_table = category_table[['Category Code','Category English Desc', 'Sub-Category Code', 'Sub-Category English Desc']] \
                .rename(columns={'Category English Desc':'Category', 'Sub-Category English Desc':'Sub-Category'})
    
designation_table = designation_table[['Designation Code', 'Description_E']] \
                    .rename(columns={'Description_E':'Designation'})
    
programs_table = programs_table[['Line 1200, 1210, 1220', 'English Description']] \
                .rename(columns={'Line 1200, 1210, 1220':'Program Code', 'English Description':'Program'})

# REVOKED/ANNULLED

In [3]:
revoked_path = r'E:\projects\canadacharities\data\raw\Revoked and Annulled'
revoked_files = [f for f in listdir(revoked_path) if isfile(join(revoked_path, f))]

revoked_path = revoked_path + '\\'
revoked_df = pd.read_csv(revoked_path + 'List of revoked & annulled charities.csv', encoding='latin-1', dtype=str)

reason_table = pd.read_csv(revoked_path + '# Reason.csv', encoding='latin-1', dtype=str)
reason_table = reason_table[['ReasonCode', 'DescriptionE']].rename(columns={'DescriptionE': 'Revoked Reason'})

revoked_df = pd.merge(revoked_df, reason_table, left_on='Reason code', right_on='ReasonCode', how='left')

revoked_df['Revoked Date'] = revoked_df['Effective date of status'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#filtering for revoked charities whose status was revoked since 2018, the year for which we have charity data.
revoked_df = revoked_df[revoked_df['Revoked Date'] >= pd.Timestamp(2018,1,1)]
revoked_df = revoked_df[['BN/Registration number', 'Legal name', 'Revoked Date', 'Revoked Reason']]

# Data setup

Let's create a new dataset to characterize each charity, using the lookup tables to fill in coded information.

In [4]:
df = pd.merge(identities_df, financial_df1, left_on='BN/Registration Number', right_on='BN/Registration number', how='outer')
df = pd.merge(df, financial_df2, left_on='BN/Registration Number', right_on='BN/Registration number', how='outer')

df['Category code'] = df['Category code'].astype(int).astype(str)
df['Sub-category code'] = df['Sub-category code'].astype(int).astype(str)

df = pd.merge(df, category_table, how='left', left_on=['Category code', 'Sub-category code'], right_on=['Category Code', 'Sub-Category Code'])
df = pd.merge(df, designation_table, left_on='Designation code', right_on='Designation Code', how='left')

df = pd.merge(df, programs_table, left_on='1200 Program Area Code', right_on='Program Code', how='left').rename(columns={'Program':'1200 Program Area'})
df = pd.merge(df, programs_table, left_on='1210 Program Area Code', right_on='Program Code', how='left').rename(columns={'Program':'1210 Program Area'})
df = pd.merge(df, programs_table, left_on='1220 Program Area Code', right_on='Program Code', how='left').rename(columns={'Program':'1220 Program Area'})

df = pd.merge(df, revoked_df, how='left', left_on='BN/Registration Number', right_on='BN/Registration number')

## Missing data

In [5]:
#checking for missing revoked data
filed_charities = len(df['BN/Registration Number'])
revoked_charities = len(revoked_df['BN/Registration number'])
missing_revoked_charities = len(set(revoked_df['BN/Registration number']) - set(df['BN/Registration Number']))
filed_revoked_charities = len(set(revoked_df['BN/Registration number']) & set(df['BN/Registration Number']))

print("There are {} charities with 2018 information filed, {} revoked charities, and {} charities who have revoked their status since 2018 but are missing 2018 information on file. \n \nThis means we only have information for {} revoked charities, or {}% of total revoked charities.".format(filed_charities, revoked_charities, missing_revoked_charities, filed_revoked_charities, round(filed_revoked_charities / revoked_charities * 100)))

There are 84548 charities with 2018 information filed, 4373 revoked charities, and 2916 charities who have revoked their status since 2018 but are missing 2018 information on file. 
 
This means we only have information for 1457 revoked charities, or 33% of total revoked charities.


# Data cleaning

In [6]:
#remove duplicate columns
df = df[df.columns.drop(list(df.filter(regex='_y')))]
df.columns = df.columns.str.rstrip('_x')

#keep only first row if charity appears more than once
df.drop_duplicates(subset='BN/Registration Number', keep = 'first', inplace = True)

#create status flag
df['Status'] = np.nan
df.loc[df['Revoked Reason'].isnull(), 'Status'] = 'Registered'
df.loc[df['Status'] != 'Registered', 'Status'] = 'Revoked'

#Bring in data dictionary with data types to assign each column
data_dic = pd.read_csv(r'E:\Projects\canadacharities\data\external\data_dictionary.csv')

#make list of columns that need to be float type
float_list = list(data_dic.loc[data_dic['data_type']=='float','column_name'])

#make list of columns that need to be bool type
bool_list = list(data_dic.loc[data_dic['data_type']=='bool','column_name'])

#change columns in float list to float
df[float_list] = df[float_list].apply(lambda x: x.str.replace(',', '')) #remove leading $ character
df[float_list] = df[float_list].apply(lambda x: x.str.replace('$', '')) #remove commas from numbers
df[float_list] = df[float_list].apply(pd.to_numeric)

#change columns in bool list to bool
df[bool_list] = df[bool_list].apply(lambda x : np.where(x == 'Y', '1', np.where(x == 'N', '0', np.nan)))

#filtering for charities only (75% of total rows)
df = df[df['Designation']=='Charitable Organization']

'''
#removing columns with more than 50% of data missing, keeping 'Revoked Date'
columns = np.append(df.columns[df.isnull().mean() <= 0.5], ['Revoked Date', 'Revoked Reason'])
df = df[columns]
'''

#putting registration date into datetime format
df['Registration date'] = df['Registration date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

#Montreal appears as both MONTREAL and MONTRÉAL - let's fix this
df.loc[df['City']=='MONTRÉAL', 'City'] = 'MONTREAL'

## Financial data

In [7]:
#create metrics for comparing charities
df['assets_vs_liabilities'] = df['4200'] / df['4350']
df['revenue_vs_expenditures'] = df['4700'] / df['5100']
df['accumulated_vs_disbursed'] = df['5500'] / df['5510']

In [8]:
#total revenue name [4700]
#total expenditure column name [5100]

revenue_list = list(data_dic.loc[data_dic['category']=='Revenue', 'column_name'])
expenditures_list = list(data_dic.loc[data_dic['category']=='Expenditures', 'column_name'])

#add suffix to revenue list to prevent saving over values
revenue_perc_list = ['{}_perc_revenue'.format(x) for x in revenue_list]
expenditures_perc_list = ['{}_perc_expenditures'.format(x) for x in expenditures_list]    

df[revenue_perc_list] = df[revenue_list].div(df['4700'], axis=0)
df[expenditures_perc_list] = df[expenditures_list].div(df['5100'], axis=0)

In [11]:
#replace infinite values with nan
df = df.replace([np.inf, -np.inf], np.nan)

In [12]:
#save output for exploratory analysis
df.to_csv('E:\Projects\canadacharities\data\interim\charity_data.csv', index=False)